<a href="https://colab.research.google.com/github/jonahcoutinho/Smartknower-miniproject/blob/main/mini_project_jonah(data_science).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This Project is a Finger digits Classifier .**
the data set has been taken from kaggle and i have used 40 images per digit, which totals to roughly 240 images
the link to the image files is:https://drive.google.com/drive/folders/1ACPJBHhUyMCmIiXVSOTWN6FVGViY8bsf?usp=sharing

import libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
!pip install opencv-python



data preprocessing

In [4]:
from skimage.transform import resize
import cv2
flat_data=[]
target=[]
datadir='/content/drive/MyDrive/fingers'
categories=['0','1','2','3','4','5']
for i in categories:
  path = os.path.join(datadir,i)
  #print(path)
  for img in os.listdir(path):
    if(img=='.ipynb_checkpoints'):
      continue
    #print(os.path.join(path,img))
    img_array=cv2.imread(os.path.join(path,img),0)
    # plt.imshow(img_array)
    # plt.show()
    img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append(np.ndarray.flatten(img_resize))
    target.append(i)

covert to DataFrame

In [5]:
df = pd.DataFrame(flat_data)
df['Target'] = target


Splitting of data


In [6]:
X=pd.DataFrame(flat_data)
y=target

In [7]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

Grid search for hyper parameter tuning

In [8]:
from sklearn.svm import SVC
model=SVC(kernel='poly',C=1).fit(xtrain,ytrain)
pred=model.predict(xtest)
from sklearn.metrics import accuracy_score
accuracy_score(ytest,pred)
from sklearn.model_selection import GridSearchCV
model=SVC()
param={'kernel':['linear','rbf','poly'],
       'C':[0.1,1,10,100]} 
grid=GridSearchCV(model,param,cv=5)
grid.fit(xtrain,ytrain)
grid.best_params_


{'C': 1, 'kernel': 'poly'}

training

In [9]:
from sklearn.svm import SVC
model=SVC(kernel='poly',C=1).fit(xtrain,ytrain)
pred=model.predict(xtest)
from sklearn.metrics import accuracy_score
accuracy_score(ytest,pred)

0.9861111111111112

Now try with random image

In [10]:
img_array=cv2.imread('/content/drive/MyDrive/fingers/5/8_5.png')
img_resize = np.ndarray.flatten(resize(img_array,(150,150,3)))
print(img_resize)

[0. 0. 0. ... 0. 0. 0.]


In [11]:
print(model.predict([img_resize]))

['5']


save model

In [14]:
import joblib
joblib.dump(model,'Fingers')



['Fingers']

Deploy


In [15]:
!pip install streamlit --quiet
#for saving model in ml we use pickle or Joblib


     |████████████████████████████████| 7.0MB 13.3MB/s 
     |████████████████████████████████| 4.6MB 53.3MB/s 
     |████████████████████████████████| 112kB 58.2MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 163kB 54.1MB/s 
     |████████████████████████████████| 122kB 50.0MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.


In [16]:

!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [24]:
%%writefile app.py
import streamlit as st
import joblib
import cv2
from skimage.transform import resize
import numpy as np
model=joblib.load('Fingers')
st.title('finger Classifier')
ip=st.text_input('Enter your message')
if ip is not None:
  img_array=cv2.imread(ip,0)
  img_resize = np.ndarray.flatten(resize(img_array,(150,150,3)))
  op=model.predict([img_resize])
  if st.button('Predict'):
    st.title(op[0])

 



Overwriting app.py


In [18]:
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://6f550ee9c165.ngrok.io'

In [20]:
model=joblib.load('Fingers')